In [32]:
import pandas as pd
import numpy as np
import geocoder

## Data Dictionary
- VendorID A code indicating the TPEP provider that provided the record.
- 1= Creative Mobile Technologies, LLC; 2= VeriFone Inc.
- tpep_pickup_datetime The date and time when the meter was engaged.
- tpep_dropoff_datetime The date and time when the meter was disengaged.
- Passenger_count The number of passengers in the vehicle.
 This is a driver-entered value.
- Trip_distance The elapsed trip distance in miles reported by the taximeter.
- Pickup_longitude Longitude where the meter was engaged.
- Pickup_latitude Latitude where the meter was engaged.
- RateCodeID The final rate code in effect at the end of the trip.
 1= Standard rate 
 2=JFK
 3=Newark
 4=Nassau or Westchester
 5=Negotiated fare
 6=Group ride
- Store_and_fwd_flag This flag indicates whether the trip record was held in vehicle
 memory before sending to the vendor, aka “store and forward,”
 because the vehicle did not have a connection to the server.
- Y= store and forward trip
- N= not a store and forward trip
- Dropoff_longitude Longitude where the meter was disengaged.
- Dropoff_ latitude Latitude where the meter was disengaged.
- Payment_type A numeric code signifying how the passenger paid for the trip.
 1= Credit card
 2= Cash
 3= No charge
 4= Dispute
 5= Unknown
 6= Voided trip
- Fare_amount The time-and-distance fare calculated by the meter.
- Extra Miscellaneous extras and surcharges. Currently, this only includes
 t he 0.50 and 1 rush hour and overnight charges.
 M TA_tax 0.50 MTA tax that is automatically triggered based on the metered
 rate in use.
- Improvement_surcharge 0.30 improvement surcharge assessed trips at the flag drop. The
- improvement surcharge began being levied in 2015.
- Tip_amount Tip amount – This field is automatically populated for credit card
 tips. Cash tips are not included.
- Tolls_amount Total amount of all tolls paid in trip.
- Total_amount The total amount charged to passengers. Does not include cash tips.

In [15]:
taxi_yellowcab_df = pd.read_csv("data/yellow_tripdata_2016-01.csv")

In [9]:
taxi_lookup = pd.read_csv("data/taxi+_zone_lookup.csv")

In [22]:
taxi_lookup.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [16]:
taxi_yellowcab_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10906858 entries, 0 to 10906857
Data columns (total 19 columns):
VendorID                 int64
tpep_pickup_datetime     object
tpep_dropoff_datetime    object
passenger_count          int64
trip_distance            float64
pickup_longitude         float64
pickup_latitude          float64
RatecodeID               int64
store_and_fwd_flag       object
dropoff_longitude        float64
dropoff_latitude         float64
payment_type             int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
dtypes: float64(12), int64(4), object(3)
memory usage: 1.5+ GB


In [17]:
taxi_yellowcab_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-01-01 00:00:00,2016-01-01 00:00:00,2,1.10,-73.990372,40.734695,1,N,-73.981842,40.732407,2,7.5,0.5,0.5,0.0,0.0,0.3,8.8
1,2,2016-01-01 00:00:00,2016-01-01 00:00:00,5,4.90,-73.980782,40.729912,1,N,-73.944473,40.716679,1,18.0,0.5,0.5,0.0,0.0,0.3,19.3
2,2,2016-01-01 00:00:00,2016-01-01 00:00:00,1,10.54,-73.984550,40.679565,1,N,-73.950272,40.788925,1,33.0,0.5,0.5,0.0,0.0,0.3,34.3
3,2,2016-01-01 00:00:00,2016-01-01 00:00:00,1,4.75,-73.993469,40.718990,1,N,-73.962242,40.657333,2,16.5,0.0,0.5,0.0,0.0,0.3,17.3
4,2,2016-01-01 00:00:00,2016-01-01 00:00:00,3,1.76,-73.960625,40.781330,1,N,-73.977264,40.758514,2,8.0,0.0,0.5,0.0,0.0,0.3,8.8


In [23]:
taxi_yellowcab_df.tail(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
10906848,2,2016-01-31 20:53:56,2016-01-31 21:05:13,1,2.84,-73.976212,40.786301,1,N,-73.977837,40.756989,1,11.5,0.5,0.5,1.50,0.00,0.3,14.30
10906849,2,2016-01-31 21:28:59,2016-01-31 22:01:58,1,7.83,-74.002953,40.750481,1,N,-73.958153,40.656689,1,29.0,0.5,0.5,5.00,0.00,0.3,35.30
10906850,2,2016-01-31 22:36:41,2016-01-31 22:45:04,1,2.50,-74.009277,40.717049,1,N,-73.994637,40.750488,1,9.5,0.5,0.5,2.16,0.00,0.3,12.96
10906851,2,2016-01-31 22:53:00,2016-01-31 22:59:37,1,1.68,-74.003578,40.750751,1,N,-74.002159,40.734909,1,7.0,0.5,0.5,1.00,0.00,0.3,9.30
10906852,2,2016-01-31 23:00:11,2016-01-31 23:12:08,1,2.65,-74.002159,40.734852,1,N,-73.999680,40.761669,1,11.0,0.5,0.5,1.00,0.00,0.3,13.30
10906853,2,2016-01-31 23:30:32,2016-01-31 23:38:18,1,2.20,-74.003578,40.751011,1,N,-73.982651,40.767509,2,8.5,0.5,0.5,0.00,0.00,0.3,9.80
10906854,1,2016-01-05 00:15:55,2016-01-05 00:16:06,1,0.00,-73.945488,40.751530,1,N,-73.945457,40.751530,2,2.5,0.5,0.5,0.00,0.00,0.3,3.80
10906855,1,2016-01-05 06:12:46,2016-03-19 20:45:50,3,1.40,-73.994240,40.766586,1,N,-73.984428,40.753922,2,7.5,0.5,0.5,0.00,0.00,0.3,8.80
10906856,1,2016-01-05 06:21:44,2016-03-28 12:54:26,1,2.10,-73.948067,40.776531,1,N,-73.978188,40.777435,1,11.5,0.0,0.5,2.45,0.00,0.3,14.75
10906857,1,2016-01-05 06:15:21,2016-01-05 06:15:36,3,0.00,-73.960938,40.758595,2,N,-73.961006,40.758583,2,52.0,0.0,0.5,0.00,5.54,0.3,58.34


In [19]:
taxi_yellowcab_df[taxi_yellowcab_df.VendorID==2].head(20)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-01-01 00:00:00,2016-01-01 00:00:00,2,1.10,-73.990372,40.734695,1,N,-73.981842,40.732407,2,7.5,0.5,0.5,0.00,0.0,0.3,8.80
1,2,2016-01-01 00:00:00,2016-01-01 00:00:00,5,4.90,-73.980782,40.729912,1,N,-73.944473,40.716679,1,18.0,0.5,0.5,0.00,0.0,0.3,19.30
2,2,2016-01-01 00:00:00,2016-01-01 00:00:00,1,10.54,-73.984550,40.679565,1,N,-73.950272,40.788925,1,33.0,0.5,0.5,0.00,0.0,0.3,34.30
3,2,2016-01-01 00:00:00,2016-01-01 00:00:00,1,4.75,-73.993469,40.718990,1,N,-73.962242,40.657333,2,16.5,0.0,0.5,0.00,0.0,0.3,17.30
4,2,2016-01-01 00:00:00,2016-01-01 00:00:00,3,1.76,-73.960625,40.781330,1,N,-73.977264,40.758514,2,8.0,0.0,0.5,0.00,0.0,0.3,8.80
5,2,2016-01-01 00:00:00,2016-01-01 00:18:30,2,5.52,-73.980118,40.743050,1,N,-73.913490,40.763142,2,19.0,0.5,0.5,0.00,0.0,0.3,20.30
6,2,2016-01-01 00:00:00,2016-01-01 00:26:45,2,7.45,-73.994057,40.719990,1,N,-73.966362,40.789871,2,26.0,0.5,0.5,0.00,0.0,0.3,27.30
9,2,2016-01-01 00:00:02,2016-01-01 00:11:08,1,3.21,-73.998344,40.723896,1,N,-73.995850,40.688400,2,11.5,0.5,0.5,0.00,0.0,0.3,12.80
10,2,2016-01-01 00:00:03,2016-01-01 00:06:19,1,0.79,-74.006149,40.744919,1,N,-73.993797,40.741440,2,6.0,0.5,0.5,0.00,0.0,0.3,7.30
11,2,2016-01-01 00:00:03,2016-01-01 00:15:49,6,2.43,-73.969330,40.763538,1,N,-73.995689,40.744251,1,12.0,0.5,0.5,3.99,0.0,0.3,17.29


- Transform lat and long to zip code

>code from: http://stackoverflow.com/questions/26914900/reverse-geocoding-with-python-geocoder


In [129]:
taxi_sample = taxi_yellowcab_df.sample(50)

In [132]:
def zip_code_extractor(df):
    """Extract the zip code from latitude and longtitude data"""

    for row in df.iterrows():
        lon = row[1]['pickup_longitude']
        lat = row[1]['pickup_latitude']
        #print(lon)
        g = geocoder.google([lat,lon], method='reverse')
        attempts = 1  # number of lookups
        print(g.postal)
        while not(g.ok) and attempts < 4:
            logging.warning('Geocoding ERROR: {}'.format(g.debug()))
            time.sleep(5)  # 2 seconds are specified in the API. If you still get errors, it's because you've reached the daily quota.
            g = geocoder.google([lat,lon], method='reverse')
            print(g.postal)
            attempts += 1
            if attempts > 3:
                logging.warning('Daily quota of google lookups exceeded.')
                break
        pcode.extend(g.postal)
        logging.info('Geocoding SUCCESS: ({},{},{})'.format(lat,lon,pcode))

In [133]:
zip_code_extractor(taxi_sample)

10038
10024
10025
10010
10014
10121
10001
10001
10003
11430
10022
10019


None
{
    "error_message": "You have exceeded your rate-limit for this API.",
    "results": [],
    "status": "OVER_QUERY_LIMIT",
    "southwest": {},
    "northeast": {},
    "locality": {},
    "postal_town": {},
    "country": {},
    "administrative_area_level_2": {},
    "street_number": {},
    "location": {},
    "neighborhood": {},
    "postal_code": {},
    "administrative_area_level_1": {},
    "route": {},
    "sublocality": {},
    "subpremise": {}
}
{
    "encoding": "utf-8",
    "location": "40.76071548461913, -73.96916961669923",
    "provider": "google",
    "status": "OVER_QUERY_LIMIT",
    "status_code": 200,
    "ok": false
}

OSM Quality
-----------
(0/-2)

Fieldnames
----------
- [ ] accuracy
- [ ] address
- [ ] bbox
- [ ] city
- [ ] client
- [ ] client_secret
- [ ] confidence
- [ ] country
- [ ] county
- [x] encoding
- [ ] housenumber
- [ ] lat
- [ ] lng
- [x] location
- [ ] neighborhood
- [ ] ok
- [ ] place
- [ ] postal
- [ ] postal_town
- [x] provider
- [ ] qu

KeyboardInterrupt: 

In [107]:
taxi_sample[["pickup_longitude","pickup_latitude"]]

,pickup_longitude,pickup_latitude
161458,-74.005119,40.719082
236960,-73.967072,40.756748
9528169,-73.990898,40.745110
1172717,-73.995491,40.759518
3210090,-73.862793,40.769051
8432533,-73.989738,40.757229
7974816,-73.966698,40.764320
9053990,-74.005966,40.742085
1376393,-73.986145,40.746700
10472774,-73.960785,40.761505
